# Numpy Financial

https://numpy.org/numpy-financial/latest/

In [1]:
import numpy as np
import numpy_financial as npf

## 1) Present Value

numpy_financial.pv(rate, nper, pmt, fv=0, when='end')

What is the present value (e.g., the initial investment) of an investment that needs to total \\$15692.93 <br> after 10 years of saving \\$100 every month? Assume the interest rate is 5% (annually) compounded monthly.

In [2]:
npf.pv(0.05/12, 10*12, -100, 15692.93)

-100.00067131625819

By convention, the negative sign represents cash flow out (i.e., money not available today). Thus, to end up with \\$15,692.93 in 10 years saving $100 a month at 5% annual interest, one’s initial deposit should also be \\$100.

If any input is array_like, pv returns an array of equal shape. Let’s compare different interest rates in the example above:

In [3]:
a = np.array((0.05, 0.04, 0.03))/12
npf.pv(a, 10*12, -100, 15692.93)

array([ -100.00067132,  -649.26771385, -1273.78633713])

So, to end up with the same \\$15692.93 under the same \\$100 per month “savings plan,” for annual interest rates of 4% and 3%, one would need initial investments of \\$649.27 and \\$1273.79, respectively.

# 2) Future Value

numpy_financial.fv(rate, nper, pmt, pv, when='end')

What is the future value after 10 years of saving \\$100 now, with an additional monthly savings of \\$100. Assume the interest rate is 5% (annually) compounded monthly?

In [4]:
npf.fv(0.05/12, 10*12, -100, -100)

15692.928894335748

By convention, the negative sign represents cash flow out (i.e. money not available today). Thus, saving \\$100 a month at 5% annual interest leads to \\$15,692.93 available to spend in 10 years.

If any input is array_like, returns an array of equal shape. Let’s compare different interest rates from the example above.

In [5]:
a = np.array((0.05, 0.06, 0.07))/12
npf.fv(a, 10*12, -100, -100)

array([15692.92889434, 16569.87435405, 17509.44688102])

# 3) Compute the interest portion of a payment.

numpy_financial.ipmt(rate, per, nper, pv, fv=0, when='end')

The total payment is made up of payment against principal plus interest.

pmt = ppmt + ipmt

What is the amortization schedule for a 1 year loan of $2500 at 8.24% interest per year compounded monthly?

In [6]:
principal = 2500.00

The ‘per’ variable represents the periods of the loan. Remember that financial equations start the period count at 1!

In [7]:
per = np.arange(1*12) + 1
ipmt = npf.ipmt(0.0824/12, per, 1*12, principal)
ppmt = npf.ppmt(0.0824/12, per, 1*12, principal)

Each element of the sum of the ‘ipmt’ and ‘ppmt’ arrays should equal ‘pmt’.

In [8]:
pmt = npf.pmt(0.0824/12, 1*12, principal)
np.allclose(ipmt + ppmt, pmt)

True

In [9]:
fmt = '{0:2d} {1:8.2f} {2:8.2f} {3:8.2f}'
for payment in per:
     index = payment - 1
     principal = principal + ppmt[index]
     print(fmt.format(payment, ppmt[index], ipmt[index], principal))

 1  -200.58   -17.17  2299.42
 2  -201.96   -15.79  2097.46
 3  -203.35   -14.40  1894.11
 4  -204.74   -13.01  1689.37
 5  -206.15   -11.60  1483.22
 6  -207.56   -10.18  1275.66
 7  -208.99    -8.76  1066.67
 8  -210.42    -7.32   856.25
 9  -211.87    -5.88   644.38
10  -213.32    -4.42   431.05
11  -214.79    -2.96   216.26
12  -216.26    -1.49    -0.00


In [10]:
interestpd = np.sum(ipmt)
np.round(interestpd, 2)

-112.98

# 4) Return the Internal Rate of Return (IRR).

numpy_financial.irr(values)

This is the “average” periodically compounded rate of return that gives a net present value of 0.0; 

The IRR is perhaps best understood through an example (illustrated using np.irr in the Examples section below). Suppose one invests 100 units and then makes the following withdrawals at regular (fixed) intervals: 39, 59, 55, 20. Assuming the ending value is 0, one’s 100 unit investment yields 173 units; however, due to the combination of compounding and the periodic withdrawals, the “average” rate of return is neither simply 0.73/4 nor (1.73)^0.25-1. Rather, it is the solution (for r) of the equation:

In [13]:
print(round(npf.irr([-100, 39, 59, 55, 20]), 5))
print(round(npf.irr([-100, 0, 0, 74]), 5))
print(round(npf.irr([-100, 100, 0, -7]), 5))
print(round(npf.irr([-100, 100, 0, 7]), 5))
print(round(npf.irr([-5, 10.5, 1, -8, 1]), 5))

0.28095
-0.0955
-0.0833
0.06206
0.0886


# 5) Modified internal rate of return (mirr)

numpy_financial.mirr(values, finance_rate, reinvest_rate)

# 6) Compute the number of periodic payments.

numpy_financial.nper(rate, pmt, pv, fv=0, when='end')

If you only had \\$150/month to pay towards the loan, how long would it take to pay-off a loan of \\$8,000 at 7% annual interest?

In [15]:
print(np.round(npf.nper(0.07/12, -150, 8000), 5))

64.07335


So, over 64 months would be required to pay off the loan.

The same analysis could be done with several different interest rates and/or payments and/or total amounts to produce an entire table.

In [16]:
npf.nper(*(np.ogrid[0.07/12: 0.08/12: 0.01/12,
                     -150   : -99    : 50    ,
                     8000   : 9001   : 1000]))

array([[[ 64.07334877,  74.06368256],
        [108.07548412, 127.99022654]],

       [[ 66.12443902,  76.87897353],
        [114.70165583, 137.90124779]]])

# 7) Net Present Value (NPV)

numpy_financial.npv(rate, values)

Returns the NPV (Net Present Value) of a cash flow series.

Consider a potential project with an initial investment of \\$40000 and projected cashflows of \\$5000, \\$8000, \\$12000 and \\$30000 at the end of each period discounted at a rate of 8% per period. To find the project’s net present value:

In [17]:
rate, cashflows = 0.08, [-40_000, 5_000, 8_000, 12_000, 30_000]
npf.npv(rate, cashflows).round(5)

3065.22267

It may be preferable to split the projected cashflow into an initial investment and expected future cashflows. In this case, the value of the initial cashflow is zero and the initial investment is later added to the future cashflows net present value:

In [18]:
initial_cashflow = cashflows[0]
cashflows[0] = 0
np.round(npf.npv(rate, cashflows) + initial_cashflow, 5)

3065.22267

# 8) PMT

numpy_financial.pmt(rate, nper, pv, fv=0, when='end')

Compute the payment against loan principal plus interest.

What is the monthly payment needed to pay off a $200,000 loan in 15 years at an annual interest rate of 7.5%?

In [19]:
npf.pmt(0.075/12, 12*15, 200000)

-1854.0247200054619

In order to pay-off (i.e., have a future-value of 0) the $200,000 obtained today, a monthly payment of $1,854.02 would be required. Note that this example illustrates usage of fv having a default value of 0.

# 9) PPMT

numpy_financial.ppmt(rate, per, nper, pv, fv=0, when='end')

Compute the payment against loan principal.

# 10) rate

numpy_financial.rate(nper, pmt, pv, fv, when='end', guess=None, tol=None, maxiter=100)

Compute the rate of interest per period.